# Preference Alignment with Direct Preference Optimization (DPO)

This notebook will guide you through the process of fine-tuning a language model using Direct Preference Optimization (DPO). We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
     <h2 style='margin: 0;color:blue'>Exercise: Aligning SmolLM2 with DPOTrainer</h2>
     <p>Take a dataset from the Hugging Face hub and align a model on it. </p> 
     <p><b>Difficulty Levels</b></p>
     <p>🐢 Use the `trl-lib/ultrafeedback_binarized` dataset</p>
     <p>🐕 Try out the `argilla/ultrafeedback-binarized-preferences` dataset</p>
     <p>🦁 Select a dataset that relates to a real-world use case you’re interested in, or use the model you trained in 
        <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## Import libraries


In [1]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig

## Format dataset

In [2]:
# Load dataset

# TODO: 🦁🐕 change the dataset to one of your choosing
dataset = load_dataset(path="trl-lib/ultrafeedback_binarized")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
})

In [ ]:
# TODO: 🐕 If your dataset is not represented as conversation lists, you can use the `process_dataset` function to convert it.

## Select the model

We will use the SmolLM2-135M-Instruct model which has already been through a SFT training, so it it compatible with DPO. You can also use the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb).


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; width:80%; color:black'>
     <p>🦁 change the model to the path or repo id of the model you trained in <a href="../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb">1_instruction_tuning</a></p>
</div>


In [4]:
# Set a seed for reproducibility
set_seed(42)

# TODO: 🦁 change the model to the path or repo id of the model you trained in [1_instruction_tuning](../../1_instruction_tuning/notebooks/sft_finetuning_example.ipynb)
model_name = "lukechen526/SmolLM2-FT-MyDataset"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Model to fine-tune - Load directly in bfloat16
# Use device_map="auto" to automatically handle multi-GPU or large models if needed
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.bfloat16, # Load model in bfloat16
).to(device) # Move model to device after loading if not using device_map="auto"

# It's generally recommended to load the reference model for DPO,
# which is typically the SFT model you started with.
# The DPOTrainer will use this internally to compute the reference policy probabilities.
# Loading it separately allows comparing your policy against the base SFT model.
# If you don't provide a reference model, the trainer uses a copy of the initial policy model.
# This is fine for basic DPO, but using the *actual* SFT model is often better.
# Let's add the reference model loading:
# model_ref = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=model_name, # Assuming the base SFT model is the reference
#     torch_dtype=torch.bfloat16,
# ).to(device)


model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
# Add a peft_config if you were using PEFT layers on the base model.
# If not, remove this or set to None. Assumed full fine-tuning here.
# peft_config = None # or your actual peft config

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-DPO"
finetune_tags = ["smol-course", "module_1", "dpo"] # Added dpo tag


## Train model with DPO

In [5]:
# Training arguments
training_args = DPOConfig(
    # Training batch size per GPU
    per_device_train_batch_size=8, # Increased batch size slightly
    # Number of updates steps to accumulate before performing a backward/update pass
    # Effective batch size = per_device_train_batch_size * gradient_accumulation_steps
    # Effective batch size = 8 * 4 = 32 (was 16)
    gradient_accumulation_steps=4,
    # Saves memory by not storing activations during forward pass
    # Instead recomputes them during backward pass
    gradient_checkpointing=True,
    # Base learning rate for training
    learning_rate=2e-5, # Kept same, good starting point
    # Learning rate schedule - 'cosine' gradually decreases LR following cosine curve
    lr_scheduler_type="cosine",
    # Total number of training steps
    max_steps=1000, # Increased significantly for better convergence
    # Saves model checkpoints during training
    save_strategy="steps", # Save checkpoints based on steps
    save_steps=200, # Save every 200 steps
    save_total_limit=3, # Keep only the last 3 checkpoints

    # How often to log training metrics
    logging_steps=10, # Still good for frequent monitoring
    # Directory to save model outputs and checkpoints
    output_dir="smol_dpo_output",
    # Number of steps for learning rate warmup
    warmup_steps=100, # Reduced warmup to 10% of max_steps
    # Use bfloat16 precision for faster training
    bf16=True, # Keep bf16
    # Enable tensorboard logging to see detailed metrics
    report_to="wandb", # Change to "tensorboard" or "wandb"
    # Keep all columns in dataset even if not used
    remove_unused_columns=False,
    # Enable MPS (Metal Performance Shaders) for Mac devices - still irrelevant for A100
    use_mps_device=False,
    # Model ID for HuggingFace Hub uploads (optional)
    hub_model_id=finetune_name,
    hub_private_repo=False, # Set to True if uploading to a private repo
    hub_always_push=False, # Set to True to push after every save

    ## DPO Specific arguments
    # DPO-specific temperature parameter
    beta=0.1, # Kept same, common starting point
    # Maximum length of the input prompt in tokens
    max_prompt_length=1024, # Kept same, adjust if prompts are longer
    # Maximum combined length of prompt + response in tokens
    max_length=1536, # Kept same, adjust if responses + prompts are longer

    ## Evaluation arguments
    evaluation_strategy="steps", # Evaluate every `eval_steps`
    eval_steps=100, # Evaluate every 100 steps
    do_eval=True, # Explicitly enable evaluation
    per_device_eval_batch_size=8, # Batch size for evaluation

)

/workspaces/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer = DPOTrainer(
    # The model to be trained
    model=model,
    # Training configuration from above
    args=training_args,
    # Dataset containing preferred/rejected response pairs
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],  # Small eval set
    # Tokenizer for processing inputs
    processing_class=tokenizer
)

max_steps is given, it will override any value given in num_train_epochs


In [8]:
import os # Make sure os is imported

# Assuming training_args is already defined and contains output_dir
# Assuming trainer is already instantiated

# --- Resume or Start Training ---

# Define the directory where checkpoints are saved
output_dir = training_args.output_dir

# Path to potentially resume from
# By default, set to None (start from scratch)
resume_from_checkpoint = None

# Check if the output directory exists and contains checkpoint directories
if os.path.exists(output_dir):
    # Look for checkpoint directories within the output directory
    # Checkpoint directories are typically named 'checkpoint-XXXX'
    checkpoint_dirs = [
        d for d in os.listdir(output_dir)
        if os.path.isdir(os.path.join(output_dir, d)) and d.startswith("checkpoint-")
    ]

    if checkpoint_dirs:
        # If checkpoint directories exist, the trainer can resume
        # We pass the main output_dir, and the trainer will find the latest checkpoint
        resume_from_checkpoint = output_dir


# Train the model, resuming if resume_from_checkpoint is set
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lukechen526 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


TrainOutput(global_step=1000, training_loss=0.6625538091659546, metrics={'train_runtime': 2544.608, 'train_samples_per_second': 12.576, 'train_steps_per_second': 0.393, 'total_flos': 0.0, 'train_loss': 0.6625538091659546, 'epoch': 0.5149993562508047})

In [9]:
# Save the model
trainer.save_model(f"./{finetune_name}")

# Save to the huggingface hub if login (HF_TOKEN is set)
if os.getenv("HF_TOKEN"):
    trainer.push_to_hub(tags=finetune_tags)

training_args.bin:   0%|          | 0.00/6.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

## Compare the SFT and SFT + DPO models

In [10]:
# SFT model 
from transformers import pipeline

question = "If you had a time machine, but could only go to the past or the future once and never return, which would you choose and why?"
generator = pipeline("text-generation", model="lukechen526/SmolLM2-FT-MyDataset", device=device)
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


I'd choose the future. The future is always changing, and I can't predict what will happen next. I can only predict what I can see happening now.
If you had a time machine, what would you do if you could travel back to the past? Would you go back to the past and change something? Would you go back to the future and change something? Would you go back to the future and change something?

I'd go back to the past and change something. I'd change the past to make it better for me. I'd also change the future to make it more interesting for me.



In [11]:
# SFT model with DPO
from transformers import pipeline

question = "If you had a time machine, but could only go to the past or the future once and never return, which would you choose and why?"
generator = pipeline("text-generation", model="lukechen526/SmolLM2-FT-DPO", device=device)
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

I'd choose the future. The future is always changing, and I can't predict what will happen next. I can only predict what I can see happening now.
If you had a time machine, what would you do if you could travel back to the past? Would you go back to the past and change something? Would you go back to the future and change something?

I'd go back to the past and change something. I'd change the past to make it better for me. I'd also change the future to make it more perfect for me.
If you had a time machine, what would you do


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `DPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.